# 데이터 전처리 코드
***
## < 사용 데이터 >

### 1. 한국관광데이터랩

- 1-1. 검색건수
- 1-2. 방문자수
- 1-3. 카드소비액
- 1-4. 연령 및 성별 방문자
- 1-5. 연령 및 성별 카드소비액
- 1-6. 숙박 일수별 방문자

### 2. 공공데이터 포털

- 2-1. 향토문화유적
- 2-2. 농어촌 체험
- 2-3. 휴양림
- 2-4. 도시공원
- 2-5. 박물관미술관
- 2-6. 길관광
- 2-7. 문화축제
- 2-8. 공연행사
- 2-9. 지역특화거리
- 2-10. 음식점
- 2-11. 주차장

### 3. 야놀자

- 3-1. 투어교통 상품
- 3-2. 취미/여가 상품
- 3-3. 체험 상품
- 3-4. 키즈 상품
- 3-5. 사업장
- 3-6. 축제/캠핑 상품
- 3-7. 테마파크 시설
- 3-8. 워터파크 시설
- 3-9. 익스트림 상품
- 3-10. 공연전시 상품

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
%matplotlib inline

import os

## 1. 한국 관광 데이터 랩



In [2]:
mega_city = sorted(os.listdir('./한국관광데이터랩/관광지출액/'))[1:]

### 1-1. 검색건수

In [3]:
search = pd.DataFrame()
for mc in mega_city :
    city_dir = sorted(os.listdir(f'./한국관광데이터랩/검색건수/{mc}'))[1:]
    
    for city in city_dir :
        file_name = sorted(os.listdir(f'./한국관광데이터랩/검색건수/{mc}/{city}'))[2]
        
        df = pd.read_csv(f'./한국관광데이터랩/검색건수/{mc}/{city}/{file_name}', encoding = 'cp949')
        df = pd.crosstab(df['기초지자체'], df['티맵카테고리중분류명'], aggfunc = lambda x : x, values = df['기초지자체 검색건수'])
        df.columns += '_검색건수'
        df = df.reset_index()
        df['기초지자체'] = mc + ' ' + df['기초지자체']
        search = pd.concat([search, df], axis = 0, ignore_index = True)
        
        del df

In [4]:
search.columns.name = None

In [5]:
search.fillna(0, inplace = True)

In [6]:
search.at[165, '기초지자체'] = '세종특별자치시'

### 1-2. 방문자수

In [7]:
visitors = pd.DataFrame()
for mc in mega_city :
    city_dir = sorted(os.listdir(f'./한국관광데이터랩/방문자수/{mc}'))[1:]
    
    for city in city_dir :
        file_name = sorted(os.listdir(f'./한국관광데이터랩/방문자수/{mc}/{city}'))[1]
        
        df = pd.read_csv(f'./한국관광데이터랩/방문자수/{mc}/{city}/{file_name}', encoding = 'cp949')
        df = pd.crosstab(df['기초지자체'], df['방문자 구분'], aggfunc = lambda x : x, values = df['방문자 수'])
        df = df[['외국인방문자(c)', '외지인방문자(b)', '현지인방문자(a)']].reset_index()
        df['기초지자체'] = mc + ' ' + df['기초지자체']
        visitors = pd.concat([visitors, df], axis = 0, ignore_index = True)
        
        del df

In [8]:
visitors.columns.name = None

In [9]:
visitors.fillna(0, inplace = True)

In [10]:
visitors.at[165, '기초지자체'] = '세종특별자치시'

### 1-3. 소비

In [11]:
consumption = pd.DataFrame()
for mc in mega_city :
    city_dir = sorted(os.listdir(f'./한국관광데이터랩/신용카드소비/{mc}'))[1:]
    
    for city in city_dir :
        try :
            file_name = sorted(os.listdir(f'./한국관광데이터랩/신용카드소비/{mc}/{city}'))[1]
        except : 
            file_name = os.listdir(f'./한국관광데이터랩/신용카드소비/{mc}/{city}')[0]
        df = pd.read_csv(f'./한국관광데이터랩/신용카드소비/{mc}/{city}/{file_name}', encoding = 'cp949')
        df['기초지자체'] = mc + ' ' + city
        df = pd.crosstab(df['기초지자체'], df['업종중분류명'], aggfunc = lambda x : x, values = df['중분류 소비액'])
        df.columns += '_신용카드소비'
        consumption = pd.concat([consumption, df.reset_index()], axis = 0, ignore_index = True)
        
        del df
        
consumption.index.name = None
consumption.columns.name = None

In [12]:
consumption.columns.name = None

In [13]:
consumption.fillna(0, inplace = True)

In [14]:
consumption.at[165, '기초지자체'] = '세종특별자치시'

### 1-4. 관광지출액

In [15]:
paid = pd.DataFrame()
for city in sorted(os.listdir(f'./한국관광데이터랩/관광지출액/'))[1:] :
    file_name = sorted(os.listdir(f'./한국관광데이터랩/관광지출액/{city}'))[-1]
    df = pd.read_csv(f'./한국관광데이터랩/관광지출액/{city}/{file_name}', encoding = 'cp949')
    df['기초지자체'] = f'{city} ' + df['기초지자체 명']
    df.drop('기초지자체 지출액 비율', axis = 1, inplace = True)
    
    paid = pd.concat([paid, df.iloc[:, 1:]], axis = 0, ignore_index = True)
    del df

### 1-5. 연령 및 성별 방문자

In [16]:
folder_dir = './한국관광데이터랩/크롤링/'

In [17]:
folders = sorted(os.listdir(folder_dir))[2:]

In [18]:
vg = pd.DataFrame()
for folder in folders :
    cities = sorted(os.listdir(folder_dir + folder))[1:]
    
    for city in cities :
        fn = sorted(os.listdir(folder_dir + folder + '/' + city))[1]
        df = pd.read_csv(folder_dir + folder + '/' + city + '/' + fn, encoding = 'cp949')
        print(city, fn)
        city = city.replace('_', ' ')
        man = df[['방문자 수(남성)']].T
        man.columns = df['방문자 연령'] + '_남성'
        man.index = [city]
        
        woman = df[['방문자 수(여성)']].T
        woman.columns = df['방문자 연령'] + '_여성'
        woman.index = [city]
        
        visitor_gender = pd.concat([man, woman], axis = 1, ignore_index = True)
        visitor_gender['기초지자체'] = city
        vg = pd.concat([vg, visitor_gender], axis = 0, ignore_index = True)

vg.columns = man.columns.tolist() + woman.columns.tolist() + ['기초지자체']

강원도_강릉시 20220902163040_방문자 성연령.csv
강원도_고성군 20220902163236_방문자 성연령.csv
강원도_동해시 20220902163415_방문자 성연령.csv
강원도_삼척시 20220902163522_방문자 성연령.csv
강원도_속초시 20220902163625_방문자 성연령.csv
강원도_양구군 20220902163717_방문자 성연령.csv
강원도_양양군 20220902163827_방문자 성연령.csv
강원도_영월군 20220902163926_방문자 성연령.csv
강원도_원주시 20220902164020_방문자 성연령.csv
강원도_인제군 20220902164110_방문자 성연령.csv
강원도_정선군 20220902164240_방문자 성연령.csv
강원도_철원군 20220902165512_방문자 성연령.csv
강원도_춘천시 20220902171046_방문자 성연령.csv
강원도_태백시 20220902171224_방문자 성연령.csv
강원도_평창군 20220902171345_방문자 성연령.csv
강원도_홍천군 20220902171459_방문자 성연령.csv
강원도_화천군 20220902172023_방문자 성연령.csv
강원도_횡성군 20220902172617_방문자 성연령.csv
경기도_가평군 20220902191504_방문자 성연령.csv
경기도_고양시덕양구 20220902191127_방문자 성연령.csv
경기도_고양시일산동구 20220902191600_방문자 성연령.csv
경기도_고양시일산서구 20220902191814_방문자 성연령.csv
경기도_과천시 20220902191926_방문자 성연령.csv
경기도_광명시 20220902192148_방문자 성연령.csv
경기도_광주시 20220902192327_방문자 성연령.csv
경기도_구리시 20220902192444_방문자 성연령.csv
경기도_군포시 20220902192603_방문자 성연령.csv
경기도_김포시 20220902192720_방문자 성연령.csv
경기도_남양주시 

아산시 20220902184823_방문자 성연령.csv
예산군 20220902184853_방문자 성연령.csv
천안시 동남구 20220902184951_방문자 성연령.csv
천안시 서북구 20220902185017_방문자 성연령.csv
청양군 20220902185112_방문자 성연령.csv
태안군 20220902185208_방문자 성연령.csv
홍성군 20220902185302_방문자 성연령.csv
괴산군 20220902181855_방문자 성연령.csv
단양군 20220902182044_방문자 성연령.csv
보은군 20220902182154_방문자 성연령.csv
영동군 20220902182255_방문자 성연령.csv
옥천군 20220902182418_방문자 성연령.csv
음성군 20220902182554_방문자 성연령.csv
제천시 20220902182708_방문자 성연령.csv
증평군 20220902182814_방문자 성연령.csv
진천군 20220902182903_방문자 성연령.csv
청주시 상당구 20220902182955_방문자 성연령.csv
청주시 서원구 20220902183116_방문자 성연령.csv
청주시 청원구 20220902183230_방문자 성연령.csv
청주시 흥덕구 20220902183323_방문자 성연령.csv
충주시 20220902183409_방문자 성연령.csv


### 1-6. 연령 및 성별 카드 소비액

In [19]:
cg = pd.DataFrame()
for folder in folders :
    cities = sorted(os.listdir(folder_dir + folder))[1:]
    
    for city in cities :
        fn = sorted(os.listdir(folder_dir + folder + '/' + city))[2]
        df = pd.read_csv(folder_dir + folder + '/' + city + '/' + fn, encoding = 'cp949')
        df.columns = ['방문자 연령', '남성소비액', '남성소비액비율', '여성소비액', '여성소비액비율']
        print(city, fn)
        city = city.replace('_', ' ')
        man = df[['남성소비액']].T
        man.columns = df['방문자 연령'] + '_남성소비액'
        man.index = [city]
        
        woman = df[['여성소비액']].T
        woman.columns = df['방문자 연령'] + '_여성소비액'
        woman.index = [city]
        
        visitor_gender = pd.concat([man, woman], axis = 1, ignore_index = True)
        visitor_gender['기초지자체'] = city
        cg = pd.concat([cg, visitor_gender], axis = 0, ignore_index = True)

cg.columns = man.columns.tolist() + woman.columns.tolist() + ['기초지자체']
cg.columns.name = None

강원도_강릉시 20220902163040_성·연령별 신용카드 소비.csv
강원도_고성군 20220902163236_성·연령별 신용카드 소비.csv
강원도_동해시 20220902163415_성·연령별 신용카드 소비.csv
강원도_삼척시 20220902163522_성·연령별 신용카드 소비.csv
강원도_속초시 20220902163625_성·연령별 신용카드 소비.csv
강원도_양구군 20220902163717_성·연령별 신용카드 소비.csv
강원도_양양군 20220902163827_성·연령별 신용카드 소비.csv
강원도_영월군 20220902163926_성·연령별 신용카드 소비.csv
강원도_원주시 20220902164020_성·연령별 신용카드 소비.csv
강원도_인제군 20220902164110_성·연령별 신용카드 소비.csv
강원도_정선군 20220902164240_성·연령별 신용카드 소비.csv
강원도_철원군 20220902165512_성·연령별 신용카드 소비.csv
강원도_춘천시 20220902171046_성·연령별 신용카드 소비.csv
강원도_태백시 20220902171224_성·연령별 신용카드 소비.csv
강원도_평창군 20220902171345_성·연령별 신용카드 소비.csv
강원도_홍천군 20220902171459_성·연령별 신용카드 소비.csv
강원도_화천군 20220902172023_성·연령별 신용카드 소비.csv
강원도_횡성군 20220902172617_성·연령별 신용카드 소비.csv
경기도_가평군 20220902191504_성·연령별 신용카드 소비.csv
경기도_고양시덕양구 20220902191127_성·연령별 신용카드 소비.csv
경기도_고양시일산동구 20220902191600_성·연령별 신용카드 소비.csv
경기도_고양시일산서구 20220902191814_성·연령별 신용카드 소비.csv
경기도_과천시 20220902191926_성·연령별 신용카드 소비.csv
경기도_광명시 20220902192148_성·연령별 신용카드 소비.csv
경기도_광

서울특별시_마포구 20220902181555_성·연령별 신용카드 소비.csv
서울특별시_서대문구 20220902181839_성·연령별 신용카드 소비.csv
서울특별시_서초구 20220902181938_성·연령별 신용카드 소비.csv
서울특별시_성동구 20220902182019_성·연령별 신용카드 소비.csv
서울특별시_성북구 20220902182107_성·연령별 신용카드 소비.csv
서울특별시_송파구 20220902182142_성·연령별 신용카드 소비.csv
서울특별시_양천구 20220902182218_성·연령별 신용카드 소비.csv
서울특별시_영등포구 20220902182255_성·연령별 신용카드 소비.csv
서울특별시_용산구 20220902182345_성·연령별 신용카드 소비.csv
서울특별시_은평구 20220902182425_성·연령별 신용카드 소비.csv
서울특별시_종로구 20220902182512_성·연령별 신용카드 소비.csv
서울특별시_중구 20220902182548_성·연령별 신용카드 소비.csv
서울특별시_중랑구 20220902182619_성·연령별 신용카드 소비.csv
세종특별자치시 20220902182714_성·연령별 신용카드 소비.csv
울산광역시_남구 20220902190137_성·연령별 신용카드 소비.csv
울산광역시_동구 20220902190351_성·연령별 신용카드 소비.csv
울산광역시_북구 20220902190419_성·연령별 신용카드 소비.csv
울산광역시_울주군 20220902190446_성·연령별 신용카드 소비.csv
울산광역시_중구 20220902190519_성·연령별 신용카드 소비.csv
인천광역시_강화군 20220902184713_성·연령별 신용카드 소비.csv
인천광역시_계양구 20220902184754_성·연령별 신용카드 소비.csv
인천광역시_남동구 20220902184830_성·연령별 신용카드 소비.csv
인천광역시_동구 20220902184910_성·연령별 신용카드 소비.csv
인천광역시_미추홀구 202209

### 1-7. 숙박 일수별 방문자

In [20]:
vs = pd.DataFrame()
for folder in folders :
    cities = sorted(os.listdir(folder_dir + folder))[1:]
    
    for city in cities :
        fn = sorted(os.listdir(folder_dir + folder + '/' + city))[3]
        df = pd.read_csv(folder_dir + folder + '/' + city + '/' + fn, encoding = 'cp949')
        print(city, fn)
        sleep = df[['방문자 수']].T
        sleep.columns = df['숙박일수 구분'] + '_방문자'
        sleep['기초지자체'] = city.replace('_', ' ')
        
        vs = pd.concat([vs, sleep], axis = 0, ignore_index  = True)
        
vs.columns.name = None

강원도_강릉시 20220902163040_숙박일수별 방문자 수.csv
강원도_고성군 20220902163236_숙박일수별 방문자 수.csv
강원도_동해시 20220902163415_숙박일수별 방문자 수.csv
강원도_삼척시 20220902163522_숙박일수별 방문자 수.csv
강원도_속초시 20220902163625_숙박일수별 방문자 수.csv
강원도_양구군 20220902163717_숙박일수별 방문자 수.csv
강원도_양양군 20220902163827_숙박일수별 방문자 수.csv
강원도_영월군 20220902163926_숙박일수별 방문자 수.csv
강원도_원주시 20220902164020_숙박일수별 방문자 수.csv
강원도_인제군 20220902164110_숙박일수별 방문자 수.csv
강원도_정선군 20220902164240_숙박일수별 방문자 수.csv
강원도_철원군 20220902165512_숙박일수별 방문자 수.csv
강원도_춘천시 20220902171046_숙박일수별 방문자 수.csv
강원도_태백시 20220902171224_숙박일수별 방문자 수.csv
강원도_평창군 20220902171345_숙박일수별 방문자 수.csv
강원도_홍천군 20220902171459_숙박일수별 방문자 수.csv
강원도_화천군 20220902172023_숙박일수별 방문자 수.csv
강원도_횡성군 20220902172617_숙박일수별 방문자 수.csv
경기도_가평군 20220902191504_숙박일수별 방문자 수.csv
경기도_고양시덕양구 20220902191127_숙박일수별 방문자 수.csv
경기도_고양시일산동구 20220902191600_숙박일수별 방문자 수.csv
경기도_고양시일산서구 20220902191814_숙박일수별 방문자 수.csv
경기도_과천시 20220902191926_숙박일수별 방문자 수.csv
경기도_광명시 20220902192148_숙박일수별 방문자 수.csv
경기도_광주시 20220902192327_숙박일수별 방문자 수.csv
경기도_구리시 202209

인천광역시_남동구 20220902184830_숙박일수별 방문자 수.csv
인천광역시_동구 20220902184910_숙박일수별 방문자 수.csv
인천광역시_미추홀구 20220902184941_숙박일수별 방문자 수.csv
인천광역시_부평구 20220902185054_숙박일수별 방문자 수.csv
인천광역시_서구 20220902185126_숙박일수별 방문자 수.csv
인천광역시_연수구 20220902185206_숙박일수별 방문자 수.csv
인천광역시_옹진군 20220902185237_숙박일수별 방문자 수.csv
인천광역시_중구 20220725231711_숙박일수별 방문자 수.csv
강진군 20220902190519_숙박일수별 방문자 수.csv
고흥군 20220902190601_숙박일수별 방문자 수.csv
곡성군 20220902190627_숙박일수별 방문자 수.csv
광양시 20220902190656_숙박일수별 방문자 수.csv
구례군 20220902190718_숙박일수별 방문자 수.csv
나주시 20220902190832_숙박일수별 방문자 수.csv
담양군 20220902190853_숙박일수별 방문자 수.csv
목포시 20220902190913_숙박일수별 방문자 수.csv
무안군 20220902190933_숙박일수별 방문자 수.csv
보성군 20220902190953_숙박일수별 방문자 수.csv
순천시 20220902191045_숙박일수별 방문자 수.csv
신안군 20220902191133_숙박일수별 방문자 수.csv
여수시 20220902191325_숙박일수별 방문자 수.csv
영광군 20220902191346_숙박일수별 방문자 수.csv
영암군 20220902191419_숙박일수별 방문자 수.csv
완도군 20220902191454_숙박일수별 방문자 수.csv
장성군 20220902191522_숙박일수별 방문자 수.csv
장흥군 20220902191554_숙박일수별 방문자 수.csv
진도군 20220902191649_숙박일수별 방문자 수.csv
함평군 20220


***
## 통계청 - Key 생성

In [21]:
population = pd.read_csv('./통계청/주민등록세대수.csv', encoding = 'cp949')

In [22]:
population = population[population['행정구역(시군구)별'].str.contains('출장소') == False]

In [23]:
mega = population[(population['행정구역(시군구)별'].str.contains('특별|광역')) | (population['행정구역(시군구)별'].str.endswith('도'))]

In [24]:
seoul = population.loc[mega.index[0] + 1 : mega.index[1] - 1]
seoul['기초지자체'] = '서울특별시 ' + seoul['행정구역(시군구)별']
seoul = seoul.iloc[:, [-1, 3]]

In [25]:
busan = population.loc[mega.index[1] + 1 : mega.index[2] - 1]
busan['기초지자체'] = '부산광역시 ' + busan['행정구역(시군구)별']
busan = busan.iloc[:, [-1, 3]]

In [26]:
dg = population.loc[mega.index[2] + 1 : mega.index[3] - 1]
dg['기초지자체'] = '대구광역시 ' + dg['행정구역(시군구)별']
dg = dg.iloc[:, [-1, 3]]

In [27]:
ic = population.loc[mega.index[3] + 1 : mega.index[4] - 1]
ic['기초지자체'] = '인천광역시 ' + ic['행정구역(시군구)별']
ic = ic.iloc[:, [-1, 3]]

In [28]:
gj = population.loc[mega.index[4] + 1 : mega.index[5] - 1]
gj['기초지자체'] = '광주광역시 ' + gj['행정구역(시군구)별']
gj = gj.iloc[:, [-1, 3]]

In [29]:
dj = population.loc[mega.index[5] + 1 : mega.index[6] - 1]
dj['기초지자체'] = '대전광역시 ' + dj['행정구역(시군구)별']
dj = dj.iloc[:, [-1, 3]]

In [30]:
us = population.loc[mega.index[6] + 1 : mega.index[7] - 1]
us['기초지자체'] = '울산광역시 ' + us['행정구역(시군구)별']
us = us.iloc[:, [-1, 3]]

In [31]:
sj = population.loc[mega.index[7] + 1 : mega.index[8] - 1]
sj['기초지자체'] = '세종특별자치시'
sj = sj.iloc[:, [-1, 3]]

In [32]:
ggd = population.loc[mega.index[8] + 1 : mega.index[9] - 1]

In [33]:
ggd.at[89, '행정구역(시군구)별'] = '수원시 장안구'
ggd.at[90, '행정구역(시군구)별'] = '수원시 권선구'
ggd.at[91, '행정구역(시군구)별'] = '수원시 팔달구'
ggd.at[92, '행정구역(시군구)별'] = '수원시 영통구'

ggd.at[94, '행정구역(시군구)별'] = '성남시 수정구'
ggd.at[95, '행정구역(시군구)별'] = '성남시 중원구'
ggd.at[96, '행정구역(시군구)별'] = '성남시 분당구'

ggd.at[99, '행정구역(시군구)별'] = '안양시 만안구'
ggd.at[100, '행정구역(시군구)별'] = '안양시 동안구'

ggd.at[108, '행정구역(시군구)별'] = '안산시 상록구'
ggd.at[109, '행정구역(시군구)별'] = '안산시 단원구'

ggd.at[111, '행정구역(시군구)별'] = '고양시 덕양구'
ggd.at[112, '행정구역(시군구)별'] = '고양시 일산동구'
ggd.at[113, '행정구역(시군구)별'] = '고양시 일산서구'

ggd.at[124, '행정구역(시군구)별'] = '용인시 처인구'
ggd.at[125, '행정구역(시군구)별'] = '용인시 기흥구'
ggd.at[126, '행정구역(시군구)별'] = '용인시 수지구'

In [34]:
ggd.drop([88, 93, 98, 107, 110, 123], axis = 0, inplace = True)
ggd['기초지자체'] = '경기도 ' + ggd['행정구역(시군구)별']
ggd = ggd.iloc[:, [-1, 3]]

In [35]:
gw = population.loc[mega.index[9] + 1 : mega.index[10] - 1]
gw['기초지자체'] = '강원도 ' + gw['행정구역(시군구)별']
gw = gw.iloc[:, [-1, 3]]

In [36]:
cb = population.loc[mega.index[10] + 1 : mega.index[11] - 1]

cb.at[162, '행정구역(시군구)별'] = '청주시 상당구'
cb.at[163, '행정구역(시군구)별'] = '청주시 서원구'
cb.at[164, '행정구역(시군구)별'] = '청주시 흥덕구'
cb.at[165, '행정구역(시군구)별'] = '청주시 청원구'

cb['기초지자체'] = '충청북도 ' + cb['행정구역(시군구)별']
cb = cb.iloc[1:, [-1, 3]]

In [37]:
cn = population.loc[mega.index[11] + 1 : mega.index[12] - 1]

cn.at[178, '행정구역(시군구)별'] = '천안시 동남구'
cn.at[179, '행정구역(시군구)별'] = '천안시 서북구'

cn['기초지자체'] = '충청남도 ' + cn['행정구역(시군구)별']
cn = cn.iloc[1:, [-1, 3]]

In [38]:
jb = population.loc[mega.index[12] + 1 : mega.index[13] - 1]

jb.at[196, '행정구역(시군구)별'] = '전주시 완산구'
jb.at[197, '행정구역(시군구)별'] = '전주시 덕진구'

jb['기초지자체'] = '전라북도 ' + jb['행정구역(시군구)별']
jb = jb.iloc[1:, [-1, 3]]

In [39]:
jn = population.loc[mega.index[13] + 1 : mega.index[14] - 1]

jn['기초지자체'] = '전라남도 ' + jn['행정구역(시군구)별']
jn = jn.iloc[:, [-1, 3]]

In [40]:
gb = population.loc[mega.index[14] + 1 : mega.index[15] - 1]

gb.at[238, '행정구역(시군구)별'] = '포항시 남구'
gb.at[239, '행정구역(시군구)별'] = '포항시 북구'

gb['기초지자체'] = '경상북도 ' + gb['행정구역(시군구)별']
gb = gb.iloc[1:, [-1, 3]]

In [41]:
gn = population.loc[mega.index[15] + 1 : mega.index[16] - 1]

gn.at[264, '행정구역(시군구)별'] = '창원시 의창구'
gn.at[265, '행정구역(시군구)별'] = '창원시 성산구'
gn.at[266, '행정구역(시군구)별'] = '창원시 마산합포구'
gn.at[267, '행정구역(시군구)별'] = '창원시 마산회원구'
gn.at[268, '행정구역(시군구)별'] = '창원시 진해구'

gn['기초지자체'] = '경상남도 ' + gn['행정구역(시군구)별']
gn = gn.iloc[1:, [-1, 3]]

In [42]:
jj = population.loc[mega.index[16] + 1 :]

jj['기초지자체'] = '제주특별자치도 ' + jj['행정구역(시군구)별']
jj = jj.iloc[:, [-1, 3]]

In [43]:
population = pd.concat([seoul, busan, dg, ic, gj, dj, us, sj, ggd, gw, cb, cn, jb, jn, gb, gn, jj], ignore_index = True)

In [44]:
standard = sorted(population["기초지자체"].tolist())

In [45]:
visitors['기초지자체'] = standard
consumption['기초지자체'] = standard
search['기초지자체'] = standard
paid['기초지자체'] = standard
vg['기초지자체'] = standard
cg['기초지자체'] = standard
vs['기초지자체'] = standard

***
## 3. 공공데이터 포털

### 3-1. 향토문화유적

In [46]:
relics = pd.read_csv('./공공데이터포털/향토문화유적.csv', encoding = 'cp949')

In [47]:
r_1 = relics[relics['소재지도로명주소'].isnull()]

In [48]:
r_1['시도명'] = r_1['소재지지번주소'].str.split(' ').str[0]
r_1['시군구명1'] = r_1['소재지지번주소'].str.split(' ').str[1]
r_1['시군구명2'] = r_1['소재지지번주소'].str.split(' ').str[2]

In [49]:
r_1_1 = r_1[r_1['시군구명2'].str.endswith('구')]

In [50]:
r_1_2 = r_1[r_1['시군구명1'].str.endswith('구')]

In [51]:
r_1_3 = r_1.query('index not in @r_1_1.index and index not in @r_1_2.index')

In [52]:
r_1_1['기초지자체'] = r_1_1['시도명'] + ' ' + r_1_1['시군구명1'] + ' ' + r_1_1['시군구명2']

In [53]:
r_1_2['기초지자체'] = r_1_2['시도명'] + ' ' + r_1_2['시군구명1']

In [54]:
r_1_4 = r_1_3[r_1_3['시도명'].str.endswith('시') | r_1_3['시도명'].str.endswith('도')]

In [55]:
r_1_4['기초지자체'] = r_1_4['시도명'] + ' ' + r_1_4['시군구명1']

In [56]:
r_1_5 = r_1_3.query('index not in @r_1_4.index')

In [57]:
r_1_5['기초지자체'] = r_1_5['관리기관명']

In [58]:
r_1 = pd.concat([r_1_1, r_1_2, r_1_4, r_1_5])

In [59]:
r_1.at[2122, '기초지자체'] = '경기도 성남시 분당구'
r_1.at[805, '기초지자체'] = '경기도 용인시 기흥구'
r_1.at[1401, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[1402, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[1403, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[1495, '기초지자체'] = '경기도 용인시 기흥구'
r_1.at[1505, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[1508, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[1526, '기초지자체'] = '경기도 용인시 기흥구'
r_1.at[1527, '기초지자체'] = '경기도 용인시 기흥구'
r_1.at[1532, '기초지자체'] = '경기도 용인시 수지구'
r_1.at[1533, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[1534, '기초지자체'] = '경기도 용인시 처인구'
r_1.at[365, '기초지자체'] = '부산광역시 강서구'
r_1.at[1254, '기초지자체'] = '제주특별자치도 제주시'

In [60]:
for idx in r_1[r_1['기초지자체'].str.contains('세종특별자치시')].index :
    r_1.at[idx, '기초지자체'] = '세종특별자치시'

In [61]:
r_2 = relics[relics['소재지지번주소'].isnull()]

In [62]:
r_2['시도명'] = r_2['소재지도로명주소'].str.split(' ').str[0]
r_2['시군구명1'] = r_2['소재지도로명주소'].str.split(' ').str[1]

In [63]:
r_2['기초지자체'] = r_2['시도명'] + ' ' + r_2['시군구명1']

In [64]:
r_3 = relics.loc[(relics['소재지지번주소'].isnull() == False) & (relics['소재지도로명주소'].isnull() == False)]

In [65]:
r_4 = r_3.query('제공기관명 in ["경기도 안양시", "경기도 안산시", "경기도 수원시", "경기도 고양시", "제주특별자치도", "경기도 성남시", "충청남도 천안시", "경상남도 창원시"]')

In [66]:
r_3 = r_3.query('제공기관명 not in ["경기도 수원시", "경기도 안양시", "경기도 안산시", "경기도 고양시", "제주특별자치도", "경기도 성남시", "충청남도 천안시", "경상남도 창원시"]')

In [67]:
r_3['기초지자체'] = r_3['제공기관명']

In [68]:
for idx in r_3[r_3['기초지자체'].isin(['광주광역시', '울산광역시'])].index :
    r_3.at[idx, '기초지자체'] = r_3['소재지도로명주소'].str.split(' ').str[0][idx] + ' ' + r_3['소재지도로명주소'].str.split(' ').str[1][idx]

In [69]:
for idx in r_3[r_3['기초지자체'].str.contains('세종특별자치시')].index :
    r_3.at[idx, '기초지자체'] = '세종특별자치시'

In [70]:
r_4['기초지자체'] = r_4['소재지도로명주소'].str.split(' ').str[0] + ' ' + r_4['소재지도로명주소'].str.split(' ').str[1] + ' ' + r_4['소재지도로명주소'].str.split(' ').str[2]

In [71]:
relics = pd.concat([r_1, r_2, r_3, r_4], ignore_index = True)

In [72]:
relics = relics.groupby('기초지자체').size().reset_index(name = '향토문화유적수')

### 3-2. 농어촌체험

In [73]:
rural = pd.read_csv('./공공데이터포털/농어촌체험휴양마을.csv', encoding = 'cp949')

In [74]:
rural_1 = rural.query('시군구명 in ["포항시", "청주시", "수원시", "안양시", "안산시", "고양시", "용인시", "제주특별자치도", "성남시", "천안시", "창원시", "전주시"]')
rural_2 = rural.query('시군구명 not in ["포항시", "청주시", "수원시", "안양시", "안산시", "고양시", "용인시", "제주특별자치도", "성남시", "천안시", "창원시", "전주시"]')

In [75]:
rural_2['기초지자체'] = rural_2['시도명'] + ' ' + rural_2['시군구명']

In [76]:
rural_1['기초지자체'] = rural_1['소재지도로명주소'].str.split(' ').str[0] + ' ' + rural_1['소재지도로명주소'].str.split(' ').str[1] + ' ' + rural_1['소재지도로명주소'].str.split(' ').str[2]

In [77]:
rural = pd.concat([rural_1, rural_2], axis = 0, ignore_index = True)

In [78]:
rural.at[663, '기초지자체'] = '세종특별자치시'
rural.at[664, '기초지자체'] = '세종특별자치시'
rural.at[665, '기초지자체'] = '세종특별자치시'

In [79]:
rural = rural.groupby('기초지자체').size().reset_index(name = '농어촌체험휴양마을수')

### 3-3. 휴양림

In [80]:
forest = pd.read_csv('./공공데이터포털/휴양림.csv', encoding = 'cp949')

In [81]:
forest['시군구명'] = forest['소재지도로명주소'].str.split(' ').str[1]

In [82]:
forest['기초지자체'] = forest['시도명'] + ' ' + forest['시군구명']

In [83]:
forest.at[25, '기초지자체'] = '경상북도 포항시 북구'
forest.at[67, '기초지자체'] = '경기도 용인시 처인구'
forest.at[100, '기초지자체'] = '충청북도 청주시 상당구'
forest.at[160, '기초지자체'] = '충청남도 천안시 동남구'
forest.at[161, '기초지자체'] = '충청북도 청주시 상당구'
forest.at[175, '기초지자체'] = '경상남도 하동군'
forest.at[179, '기초지자체'] = '세종특별자치시'
forest.at[91, '기초지자체'] = '경상남도 의령군'

In [84]:
forest = forest.groupby('기초지자체')['수용인원수'].sum().reset_index(name = '휴양림수용인원수')

### 3-4. 도시공원

In [85]:
park = pd.read_csv('./공공데이터포털/도시공원정보.csv', encoding = 'cp949')

In [86]:
park1 = park[park['소재지도로명주소'].isnull()]

In [87]:
park2 = park.query('index not in @park1.index')

In [88]:
park1['시도명'] = park1['소재지지번주소'].str.split(' ').str[0]
park1['시군구명1'] = park1['소재지지번주소'].str.split(' ').str[1]
park1['시군구명2'] = park1['소재지지번주소'].str.split(' ').str[2]

In [89]:
park1['기초지자체'] = park1['시도명'] + ' ' + park1['시군구명1']

In [90]:
pp1 = ['경기도 용인시', '경기도 고양시', '경기도 성남시', '경기도 수원시', '경기도 안산시', '경상남도 창원시', '경상북도 포항시', '수원시 권선구', 
     '전라북도 전주시', '충청남도 천안시', '충청북도 청주시', '경기도 안양시']

In [91]:
pp2 = ['전라북도 도통동', '충청남도 결성면', '충청남도 대산읍', '충청남도 부석면', '대전광역시 신흥동', '상계 산3-1','서울특별시 구로구개봉동',
       '서울특별시 남가좌동', '서울특별시 면목동', '서울특별시 성북동', '서울특별시 은평구구산동', '서울특별시 은평구응암동','서울특별시 은평구진관동',
       '서울특별시 은평구진관동13-2,', '서울특별시 정릉동', '영월군 영월읍', '울산광역시 온양읍', '전라남도 영암1공구']

In [92]:
park1_1 = park1.query('기초지자체 in @pp1')

In [93]:
park1_1['기초지자체'] = park1_1['시도명'] + ' ' + park1_1['시군구명1'] + ' ' + park1_1['시군구명2']

In [94]:
park1_1.at[1001, '기초지자체'] = '경기도 수원시 권선구'

In [95]:
park1_2 = park1.query('기초지자체 in @pp2')

In [96]:
park1_2['기초지자체'] = park1_2['제공기관명']

In [97]:
park1_2.at[1881, '기초지자체'] = '충청남도 서산시'
park1_2.at[7049, '기초지자체'] = '충청남도 홍성군'
park1_2.at[7967, '기초지자체'] = '충청남도 서산시'
park1_2.at[7968, '기초지자체'] = '충청남도 서산시'

In [98]:
park1_3 = park1.query('기초지자체 not in @pp1 and 기초지자체 not in @pp2')

In [99]:
park1 = pd.concat([park1_1, park1_2, park1_3], ignore_index = True)

In [100]:
park2['시도명'] = park2['소재지도로명주소'].str.split(' ').str[0]
park2['시군구명1'] = park2['소재지도로명주소'].str.split(' ').str[1]
park2['시군구명2'] = park2['소재지도로명주소'].str.split(' ').str[2]

In [101]:
park2['기초지자체'] = park2['시도명'] + ' ' + park2['시군구명1']

In [102]:
park2_1 = park2[park2['기초지자체'].isnull()]

In [103]:
park2_1['기초지자체'] = park2['제공기관명']

In [104]:
park2_2 = park2.query('기초지자체 in @standard')

In [105]:
park2_3 = park2.query('index not in @park2_1.index and index not in @park2_2.index')

In [106]:
park2_4 = park2_3[park2_3['기초지자체'].str.contains('로|길')]

In [107]:
park2_4['기초지자체'] = park2_4['제공기관명']

In [108]:
park2_5 = park2_3[park2_3['기초지자체'].str.contains('로|길') == False]

In [109]:
park2_6 = park2_5.query('기초지자체 in @pp1')

In [110]:
park2_6['기초지자체'] = park2_5['시도명'] + ' ' + park2_5['시군구명1'] + ' ' + park2_5['시군구명2']

In [111]:
park2_6.at[419, '기초지자체'] = '경기도 안양시 만안구'
park2_6.at[538, '기초지자체'] = '경기도 수원시 팔달구'
park2_6.at[1759, '기초지자체'] = '경기도 용인시 수지구'
park2_6.at[4630, '기초지자체'] = '경기도 안양시 동안구'

In [112]:
park2_7 = park2_5.query('기초지자체 not in @pp1')

In [113]:
park2_7['기초지자체'] = park2_7['제공기관명']

In [114]:
for idx in park2_7.query('기초지자체 == "세종특별자치시"').index :
    park2_7.at[idx, '기초지자체'] = '세종특별자치시'

In [115]:
park = pd.concat([park1, park2_1, park2_2, park2_4, park2_6, park2_7], ignore_index = True)

In [116]:
park = park.query('공원구분 != "묘지공원"')

In [117]:
park = park.groupby('기초지자체').size().reset_index(name = '도시공원수')

### 3-5. 박물관

In [118]:
ml = pd.read_csv('./공공데이터포털/박물관미술관정보.csv', encoding = 'cp949')

In [119]:
ml['소재지도로명주소'] = ml['소재지도로명주소'].str.replace('  ', ' ')

In [120]:
ml['시도명'] = ml['소재지도로명주소'].str.split(' ').str[0]
ml['시군구명1'] = ml['소재지도로명주소'].str.split(' ').str[1]
ml['시군구명2'] = ml['소재지도로명주소'].str.split(' ').str[2]

In [121]:
ml['기초지자체'] = ml['시도명'] + ' ' + ml['시군구명1']

In [122]:
ml1 = ml.query('기초지자체 in @pp1')

In [123]:
ml1['기초지자체'] = ml1['시도명'] + ' ' + ml1['시군구명1'] + ' ' + ml1['시군구명2']

In [124]:
ml1.at[1748, '기초지자체'] = '경기도 용인시 처인구'

In [125]:
ml2 = ml.query('기초지자체 not in @pp1')

In [126]:
ml3 = ml2.query('기초지자체 in @standard')

In [127]:
ml4 = ml2.query('기초지자체 not in @standard')

In [128]:
ml4.at[1759, '기초지자체'] = '서울특별시 관악구'

In [129]:
for idx in ml4.index[:-1] :
    ml4.at[idx, '기초지자체'] = '세종특별자치시'

In [130]:
ml = pd.concat([ml1, ml3, ml4], ignore_index = True)

In [131]:
ml = ml.groupby('기초지자체').size().reset_index(name = '박물관미술관수')

### 3-6. 길관광정보

In [132]:
road = pd.read_csv('./공공데이터포털/길관광정보.csv', encoding = 'cp949')

In [133]:
road1 = road.drop(['종료지점명', '종료지점소재지도로명주소', '종료지점소재지지번주소'], axis = 1)
road2 = road.drop(['시작지점명', '시작지점도로명주소', '시작지점소재지지번주소'], axis = 1)

In [134]:
road1.rename(columns = {'시작지점명' : '지점명', '시작지점도로명주소' : '소재지도로명주소', '시작지점소재지지번주소' : '소재지지번주소'}, inplace = True)
road2.rename(columns = {'종료지점명' : '지점명', '종료지점소재지도로명주소' : '소재지도로명주소', '종료지점소재지지번주소' : '소재지지번주소'}, inplace = True)

In [135]:
road = pd.concat([road1, road2], ignore_index = True)

In [136]:
road1 = road[road['소재지도로명주소'].isnull()]

In [137]:
road2 = road[road['소재지도로명주소'].isnull() == False]

In [138]:
road1['시도명'] = road1['소재지지번주소'].str.split(' ').str[0]
road1['시군구명1'] = road1['소재지지번주소'].str.split(' ').str[1]
road1['시군구명2'] = road1['소재지지번주소'].str.split(' ').str[2]

In [139]:
road1['기초지자체'] = road1['시도명'] + ' ' + road1['시군구명1']

In [140]:
road1_2 = road1.query('기초지자체 in @standard')

In [141]:
road1_3 = road1.query('기초지자체 not in @standard')

In [142]:
road1_4 = road1_3.query('기초지자체 in @pp1')

In [143]:
road1_4['기초지자체'] = road1_4['시도명'] + ' ' + road1_4['시군구명1'] + ' ' + road1_4['시군구명2']

In [144]:
road1_5 = road1_3.query('기초지자체 not in @pp1')

In [145]:
road1_5['기초지자체'] = road1_5['제공기관명']

In [146]:
road2['소재지도로명주소'] = road2['소재지도로명주소'].str.replace('대전 ', '대전광역시 ')

In [147]:
road2['시도명'] = road2['소재지도로명주소'].str.split(' ').str[0]
road2['시군구명1'] = road2['소재지도로명주소'].str.split(' ').str[1]
road2['시군구명2'] = road2['소재지도로명주소'].str.split(' ').str[2]

In [148]:
road2['기초지자체'] = road2['시도명'] + ' ' + road2['시군구명1']

In [149]:
road2_1 = road2.query('기초지자체 in @standard')

In [150]:
road2_2 = road2.query('기초지자체 not in @standard')

In [151]:
road2_2['기초지자체'] = road2_2['시도명'] + ' ' + road2_2['시군구명1'] + ' ' + road2_2['시군구명2']

In [152]:
road2_2.at[2270, '기초지자체'] = '울산광역시 울주군'

In [153]:
road2_3 = road2_2.query('기초지자체 in @standard')

In [154]:
road2_4 = road2_2.query('기초지자체 not in @standard')

In [155]:
road2_4['기초지자체'] = road2_4['제공기관명']

In [156]:
for idx in road2_4.query('기초지자체 == "세종특별자치시"').index :
    road2_4.at[idx, '기초지자체'] = '세종특별자치시'

In [157]:
road2_4.at[910, '기초지자체'] = '경기도 양주시'

In [158]:
road = pd.concat([road1_2, road1_4, road1_5, road2_1, road2_3, road2_4], ignore_index = True)

In [159]:
road = road.groupby('기초지자체').size().reset_index(name = '관광길수')

### 3-7. 문화축제

In [160]:
festival = pd.read_csv('./공공데이터포털/문화축제.csv', encoding = 'cp949')

In [161]:
festival = festival.query('축제종료일자 > "2022-06-01"')

In [162]:
festival1 = festival[festival['소재지도로명주소'].isnull()]

In [163]:
festival1['시도명'] = festival1['소재지지번주소'].str.split(' ').str[0]
festival1['시군구명1'] = festival1['소재지지번주소'].str.split(' ').str[1]
festival1['시군구명2'] = festival1['소재지지번주소'].str.split(' ').str[2]

In [164]:
festival1['기초지자체'] = festival1['시도명'] + ' ' + festival1['시군구명1']# + ' ' + festival1['시군구명2']

In [165]:
festival1_1 = festival1.query('기초지자체 in ["경북 봉화군", "경상남도 창원시", "경기도 고촌읍", "경기도 수원시", "경기도 안산시"]')

In [166]:
festival1_1.at[239, '기초지자체'] = '경상북도 봉화군'
festival1_1.at[240, '기초지자체'] = '경상북도 봉화군'
festival1_1.at[290, '기초지자체'] = '경상남도 창원시 마산합포구'
festival1_1.at[532, '기초지자체'] = '경기도 김포시'
festival1_1.at[562, '기초지자체'] = '경기도 수원시 팔달구'

In [167]:
festival1_2 = festival1.query('기초지자체 not in ["경북 봉화군", "경상남도 창원시", "경기도 고촌읍", "경기도 수원시", "경기도 안산시"]')

In [168]:
festival2 = festival[festival['소재지도로명주소'].isnull() == False]

In [169]:
festival2['시도명'] = festival2['소재지도로명주소'].str.split(' ').str[0]
festival2['시군구명1'] = festival2['소재지도로명주소'].str.split(' ').str[1]
festival2['시군구명2'] = festival2['소재지도로명주소'].str.split(' ').str[2]

In [170]:
festival2['기초지자체'] = festival2['시도명'] + ' ' + festival2['시군구명1']# + ' ' + festival2['시군구명2']

In [171]:
festival2_1 = festival2[festival2['기초지자체'].isnull()]

In [172]:
festival2_1.at[394, '기초지자체'] = '인천광역시 남동구'
festival2_1.at[522, '기초지자체'] = '경기도 수원시 팔달구'
festival2_1.at[886, '기초지자체'] = '충청남도 예산군'

In [173]:
festival2_2 = festival2.query('기초지자체 in @standard')

In [174]:
festival2_3 = festival2.query('기초지자체 not in @standard and index not in @festival2_1.index')

In [175]:
festival2_4 = festival2_3.query('기초지자체 in @pp1')

In [176]:
festival2_4['기초지자체'] = festival2_4['시도명'] + ' ' + festival2_4['시군구명1'] + ' ' + festival2_4['시군구명2']

In [177]:
festival2_5 = festival2_3.query('기초지자체 not in @pp1')

In [178]:
festival2_5.at[164, '기초지자체'] = '경상북도 울릉군'
festival2_5.at[165, '기초지자체'] = '경상북도 울릉군'
festival2_5.at[166, '기초지자체'] = '경상북도 울릉군'

In [179]:
festival = pd.concat([festival1_1, festival1_2, festival2_1, festival2_2, festival2_4, festival2_5], ignore_index = True)

In [180]:
festival = festival.groupby('기초지자체').size().reset_index(name = '문화축제수')

### 3-8. 공연행사

In [181]:
contest = pd.read_csv('./공공데이터포털/공연행사정보.csv', encoding = 'cp949')

In [182]:
contest = contest.query('행사종료일자 > "2022-06-01"')

In [183]:
contest['시도명'] = contest['소재지지번주소'].str.split(' ').str[0]
contest['시군구명1'] = contest['소재지지번주소'].str.split(' ').str[1]
contest['시군구명2'] = contest['소재지지번주소'].str.split(' ').str[2]

In [184]:
contest['기초지자체'] = contest['시도명'] + ' ' + contest['시군구명1']

In [185]:
contest1 = contest[contest['기초지자체'].isnull()]

In [186]:
contest1['시도명'] = contest1['소재지도로명주소'].str.split(' ').str[0]
contest1['시군구명1'] = contest1['소재지도로명주소'].str.split(' ').str[1]
contest1['시군구명2'] = contest1['소재지도로명주소'].str.split(' ').str[2]

In [187]:
contest1['기초지자체'] = contest1['시도명'] + ' ' + contest1['시군구명1']

In [188]:
contest1_1 = contest1.query('기초지자체 in @standard')

In [189]:
contest1_2 = contest1.query('기초지자체 not in @standard')

In [190]:
contest1_2['기초지자체'] = contest1_2['시도명'] + ' ' + contest1_2['시군구명1'] + ' ' + contest1_2['시군구명2']

In [191]:
contest2 = contest[contest['기초지자체'].isnull() == False]

In [192]:
contest2_1 = contest2.query('기초지자체 in @standard')

In [193]:
contest2_2 = contest2.query('기초지자체 not in @standard')

In [194]:
contest2_2['기초지자체'] = contest2_2['시도명'] + ' ' + contest2_2['시군구명1'] + ' ' + contest2_2['시군구명2']

In [195]:
contest2_2.at[347, '기초지자체'] = '경상북도 봉화군'
contest2_2.at[4833, '기초지자체'] = '전라남도 구례군'
contest2_2.at[8563, '기초지자체'] = '강원도 인제군'
contest2_2.at[8564, '기초지자체'] = '강원도 인제군'
contest2_2.at[8565, '기초지자체'] = '강원도 인제군'

In [196]:
contest = pd.concat([contest1_1, contest1_2, contest2_1, contest2_2], ignore_index = True)

In [197]:
contest = contest.groupby('기초지자체').size().reset_index(name = '공연행사수')

In [198]:
del contest1, contest1_1, contest1_2, contest2, contest2_1, contest2_2

### 3-9. 지역특화거리

In [199]:
street = pd.read_csv('./공공데이터포털/지역특화거리.csv', encoding = 'cp949')

In [200]:
street['시도명'] = street['소재지도로명'].str.split(' ').str[0]
street['시군구명1'] = street['소재지도로명'].str.split(' ').str[1]
street['시군구명2'] = street['소재지도로명'].str.split(' ').str[2]

In [201]:
street['기초지자체'] = street['시도명'] + ' ' + street['시군구명1']

In [202]:
street1 = street.query('기초지자체 in @standard')

In [203]:
street2 = street.query('기초지자체 not in @standard')

In [204]:
street2_1 = street2.query('기초지자체 in @pp1')

In [205]:
street2_1['기초지자체'] = street2_1['시도명'] + ' ' + street2_1['시군구명1'] + ' ' + street2_1['시군구명2']

In [206]:
street2_2 = street2.query('기초지자체 not in @pp1')

In [207]:
street2_2['기초지자체'] = street2_2['제공기관명']

In [208]:
street = pd.concat([street1, street2_1, street2_2], ignore_index = True)

In [209]:
street = street.groupby('기초지자체').size().reset_index(name = '특화거리수')

### 3-10. 음식점

In [210]:
food = pd.read_csv("./공공데이터포털/fulldata_07_24_04_P_일반음식점.csv", encoding = 'cp949')

In [211]:
food = food[['영업상태명', '소재지전체주소', '도로명전체주소', '업태구분명']]

In [212]:
food = food[food['영업상태명'] != '폐업']

In [213]:
food1 = food[food['도로명전체주소'].isnull()]

In [214]:
food1['기초지자체'] = food1['소재지전체주소'].str.split(' ').str[0] + ' ' + food1['소재지전체주소'].str.split(' ').str[1]

In [215]:
food1_1 = food1.query('기초지자체 in @standard')

In [216]:
food1_2 = food1.query('기초지자체 not in @standard')

In [217]:
food1_2['기초지자체'] = food1_2['소재지전체주소'].str.split(' ').str[0] + ' ' + food1_2['소재지전체주소'].str.split(' ').str[1] + ' ' + food1_2['소재지전체주소'].str.split(' ').str[2]

In [218]:
food2 = food[food['도로명전체주소'].isnull() == False]

In [219]:
food2['기초지자체'] = food2['도로명전체주소'].str.split(' ').str[0] + ' ' + food2['도로명전체주소'].str.split(' ').str[1]

In [220]:
food2_1 = food2.query('기초지자체 in @standard')

In [221]:
food2_2 = food2.query('기초지자체 not in @standard')

In [222]:
food2_2['기초지자체'] = food2_2['도로명전체주소'].str.split(' ').str[0] + ' ' + food2_2['도로명전체주소'].str.split(' ').str[1] + ' ' + food2_2['도로명전체주소'].str.split(' ').str[2]

In [223]:
food = pd.concat([food1_1, food1_2, food2_1, food2_2], ignore_index = True)

In [224]:
for idx in food[food['기초지자체'].str.contains('세종')].index :
    food.at[idx, '기초지자체'] = '세종특별자치시'

In [225]:
food = food.groupby('기초지자체').size().reset_index(name = '음식점수')

In [226]:
del food1, food1_1, food1_2, food2, food2_1, food2_2

### 3-11. 주차장

In [227]:
parking = pd.read_csv('./공공데이터포털/주차장.csv', encoding = 'cp949')

In [228]:
parking = parking[['주차장명', '주차장구분', '주차장유형', '소재지도로명주소', '소재지지번주소', '주차구획수', '제공기관명']]

In [229]:
parking1 = parking[parking['소재지도로명주소'].isnull()]

In [230]:
parking1_1 = parking1.query('제공기관명 in @standard')

In [231]:
parking1_2 = parking1.query('제공기관명 not in @standard')

In [232]:
parking1_2['기초지자체'] = parking1_2['소재지지번주소'].str.split(' ').str[0] + ' ' + parking1_2['소재지지번주소'].str.split(' ').str[1]

In [233]:
parking1_3 = parking1_2.query('기초지자체 in @standard')

In [234]:
parking1_4 = parking1_2.query('기초지자체 not in @standard')

In [235]:
parking1_4['기초지자체'] = parking1_4['소재지지번주소'].str.split(' ').str[0] + ' ' + parking1_4['소재지지번주소'].str.split(' ').str[1] + ' ' + parking1_4['소재지지번주소'].str.split(' ').str[2]

In [236]:
parking1_5 = parking1_4.query('기초지자체 in @standard')

In [237]:
parking1_6 = parking1_4.query('기초지자체 not in @standard')

In [238]:
parking1_6.at[1938, '기초지자체'] = '경상북도 포항시'
parking1_6.at[10030, '기초지자체'] = '인천광역시 계양구'
parking1_6.at[12285, '기초지자체'] = '인천광역시 계양구'
parking1_6.at[12286, '기초지자체'] = '인천광역시 계양구'
parking1_6.at[12287, '기초지자체'] = '인천광역시 계양구'
parking1_6.at[12288, '기초지자체'] = '인천광역시 계양구'

In [239]:
parking2 = parking[parking['소재지도로명주소'].isnull() == False]

In [240]:
parking2_1 = parking2.query('제공기관명 in @standard')

In [241]:
parking2_2 = parking2.query('제공기관명 not in @standard')

In [242]:
parking2_2['기초지자체'] = parking2_2['소재지도로명주소'].str.split(' ').str[0] + ' ' + parking2_2['소재지도로명주소'].str.split(' ').str[1]

In [243]:
parking2_3 = parking2_2.query('기초지자체 in @standard')

In [244]:
parking2_4 = parking2_2.query('기초지자체 not in @standard')

In [245]:
parking2_4['기초지자체'] = parking2_4['소재지도로명주소'].str.split(' ').str[0] + ' ' + parking2_4['소재지도로명주소'].str.split(' ').str[1] + ' ' + parking2_4['소재지도로명주소'].str.split(' ').str[2]

In [246]:
parking2_4.at[11700, '기초지자체'] = '세종특별자치시'
parking2_4.at[11714, '기초지자체'] = '세종특별자치시'

In [247]:
for idx in parking2_4[parking2_4['기초지자체'].str.contains('세종')].index :
    parking2_4.at[idx, '기초지자체'] = '세종특별자치시'

In [248]:
parking2_4.at[1931, '기초지자체'] = '경상북도 포항시 북구'
parking2_4.at[4671, '기초지자체'] = '경기도 용인시 수지구'
parking2_4.at[5871, '기초지자체'] = '경기도 성남시 분당구'
parking2_4.at[8391, '기초지자체'] = '경상남도 창원시 의창구'
parking2_4.at[8834, '기초지자체'] = '경상남도 창원시 의창구'
parking2_4.at[8835, '기초지자체'] = '경상남도 창원시 의창구'
parking2_4.at[13633, '기초지자체'] = '경기도 수원시 권선구'

In [249]:
parking = pd.concat([parking1_1, parking1_3, parking1_5, parking1_6, parking2_1, parking2_3, parking2_4], ignore_index = True)

In [250]:
parking1_1['기초지자체'] = parking1_1['제공기관명']

In [251]:
parking = parking.groupby('기초지자체')['주차구획수'].sum().reset_index(name = '주차구획수')

***
## 4. 야놀자 

In [252]:
def convert_addr(x) :
    
    if '강원 ' in x :
        x = x.replace('강원 ', '강원도 ')
        return x

    elif '경기 ' in x :
        x = x.replace('경기 ', '경기도 ')
        return x
    
    elif '광주 ' in x :
        x = x.replace('광주 ', '광주광역시 ')
        return x
    
    elif '경남 ' in x :
        x = x.replace('경남 ', '경상남도 ')
        return x
    
    elif '경북 ' in x :
        x = x.replace('경북 ', '경상북도 ')
        return x
    
    elif '울산 ' in x :
        x = x.replace('울산 ', '울산광역시 ')
        return x
    
    elif '울산시 ' in x :
        x = x.replace('울산시 ', '울산광역시 ')
        return x
    
    elif ('대구 ' in x and '해운대구' not in x):
        x = x.replace('대구 ', '대구광역시 ')
        return x
    
    elif '대구시 ' in x :
        x = x.replace('대구시 ', '대구광역시 ')
        return x
    
    elif '대전 ' in x :
        x = x.replace('대전 ', '대전광역시 ')
        return x
    
    elif '대전시 ' in x :
        x = x.replace('대전시 ', '대전광역시 ')
        return x
    
    elif '부산 ' in x :
        x = x.replace('부산 ', '부산광역시 ')
        return x
    
    elif '부산시 ' in x :
        x = x.replace('부산시 ', '부산광역시 ')
        return x
    
    elif '서울 ' in x :
        x = x.replace('서울 ', '서울특별시 ')
        return x
    
    elif '서울시 ' in x :
        x = x.replace('서울시 ', '서울특별시 ')
        return x
    
    elif '인천 ' in x :
        x = x.replace('인천 ', '인천광역시 ')
        return x
    
    elif '인천시 ' in x :
        x = x.replace('인천시 ', '인천광역시 ')
        return x
    
    elif '전남 ' in x :
        x = x.replace('전남 ', '전라남도 ')
        return x
    
    elif '전북 ' in x :
        x = x.replace('전북 ', '전라북도 ')
        return x
    
    elif '제주 ' in x :
        x = x.replace('제주 ', '제주특별자치도 ')
        return x
    
    elif '제주도 ' in x :
        x = x.replace('제주도 ', '제주특별자치도 ')
        return x
    
    elif '충남 ' in x :
        x = x.replace('충남 ', '충청남도 ')
        return x
    
    elif '충북 ' in x :
        x = x.replace('충북 ', '충청북도 ')
        return x
    
    else :
        return x

### 4-1. 투어상품

In [253]:
ynj_tour = pd.read_csv('./야놀자/YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202206.csv')

In [254]:
ynj_tour.drop([0, 891], axis = 0, inplace = True)

In [255]:
ynj_tour.at[15, 'TOUR_ND_TRNSPORT_GOODS_ADDR'] = '강원도 삼척시'
ynj_tour.at[505, 'TOUR_ND_TRNSPORT_GOODS_ADDR'] = '서울특별시 영등포구'
ynj_tour.at[893, 'TOUR_ND_TRNSPORT_GOODS_ADDR'] = '제주도 서귀포시'

In [256]:
ynj_tour['TOUR_ND_TRNSPORT_GOODS_ADDR'] = ynj_tour['TOUR_ND_TRNSPORT_GOODS_ADDR'].apply(convert_addr)

In [257]:
ynj_tour['시도명'] = ynj_tour['TOUR_ND_TRNSPORT_GOODS_ADDR'].str.split(' ').str[0]

In [258]:
ynj_tour['시군구명'] = ynj_tour['TOUR_ND_TRNSPORT_GOODS_ADDR'].str.split(' ').str[1]

In [259]:
ynj_tour['기초지자체'] = ynj_tour['시도명'] + ' ' + ynj_tour['시군구명']

In [260]:
ynj_tour1 = ynj_tour.query('기초지자체 in @standard')

In [261]:
ynj_tour2 = ynj_tour.query('기초지자체 not in @standard')

In [262]:
ynj_tour2_1 = ynj_tour2.query('기초지자체 in @pp1')

In [263]:
ynj_tour2_1['기초지자체'] = ynj_tour2_1['시도명'] + ' ' + ynj_tour2_1['시군구명'] + ' ' + ynj_tour2_1['TOUR_ND_TRNSPORT_GOODS_ADDR'].str.split(' ').str[2]

In [264]:
ynj_tour = pd.concat([ynj_tour1, ynj_tour2_1], ignore_index = True)

In [265]:
ynj_tour = ynj_tour.groupby('기초지자체').size().reset_index(name = '투어상품수')

In [266]:
ynj_tour.at[92, '투어상품수'] = 82
ynj_tour.at[93, '투어상품수'] = 59

In [267]:
del ynj_tour1, ynj_tour2, ynj_tour2_1

### 4-2. 취미상품

In [268]:
hobby = pd.read_csv('./야놀자/YN_REGNAL_HOBBY_ND_LSR_GOODS_202206.csv')

In [269]:
hobby['AREA_NM'] = hobby['AREA_NM'].apply(convert_addr)

In [270]:
hobby1 = hobby.query('AREA_NM in @standard')

In [271]:
hobby2 = hobby.query('AREA_NM not in @standard')

In [272]:
hobby2_1 = hobby2.query('AREA_NM in @pp1')

In [273]:
hobby2_1['AREA_NM'] = hobby2_1['AREA_NM'] + ' ' + hobby2_1['HOBBY_ND_LSR_GOODS_ADDR'].str.split(' ').str[2]

In [274]:
hobby2_2 = hobby2.query('AREA_NM not in @pp1')

In [275]:
hobby2_2.at[117, 'AREA_NM'] = '서울특별시 강남구'
hobby2_2.at[118, 'AREA_NM'] = '서울특별시 강남구'
hobby2_2.at[135, 'AREA_NM'] = '서울특별시 관악구'
hobby2_2.at[150, 'AREA_NM'] = '서울특별시 마포구'
hobby2_2.at[151, 'AREA_NM'] = '서울특별시 마포구'
hobby2_2.at[152, 'AREA_NM'] = '서울특별시 마포구'
hobby2_2.at[351, 'AREA_NM'] = '제주특별자치도 제주시'

In [276]:
hobby = pd.concat([hobby1, hobby2_1, hobby2_2], ignore_index = True)

In [277]:
hobby['기초지자체'] = hobby['AREA_NM']

In [278]:
hobby = hobby.groupby('기초지자체').size().reset_index(name = '취미상품수')

In [279]:
del hobby1, hobby2, hobby2_1, hobby2_2

### 4-3. 체험상품

In [280]:
exp = pd.read_csv('./야놀자/YN_REGNAL_EXPRN_GOODS_202206.csv')

In [281]:
exp.drop(3206, axis = 0, inplace = True)

In [282]:
exp['EXPRN_GOODS_ADDR'].fillna('-', inplace = True)

In [283]:
exp['EXPRN_GOODS_ADDR'] = exp['EXPRN_GOODS_ADDR'].str.replace('해운대구광역시 ', '해운대구 ')

In [284]:
exp['EXPRN_GOODS_ADDR'] = exp['EXPRN_GOODS_ADDR'].apply(convert_addr)

In [285]:
exp['시도명'] = exp['EXPRN_GOODS_ADDR'].str.split(' ').str[0]
exp['시군구명'] = exp['EXPRN_GOODS_ADDR'].str.split(' ').str[1]

In [286]:
exp['기초지자체'] = exp['시도명'] + ' ' + exp['시군구명']

In [287]:
exp1 = exp.query('기초지자체 in @standard')

In [288]:
exp2 = exp.query('기초지자체 not in @standard')

In [289]:
exp2_1 = exp2.query('기초지자체 in @pp1')

In [290]:
exp2_1['기초지자체'] = exp2_1['시도명'] + ' ' + exp2_1['시군구명'] + ' ' + exp2_1['EXPRN_GOODS_ADDR'].str.split(' ').str[2]

In [291]:
exp2_2 = exp2.query('기초지자체 not in @pp1')

In [292]:
exp2_3 = exp2_2.query('시도명 == "세종특별자치시"')

In [293]:
exp2_3['기초지자체'] = '세종특별자치시'

In [294]:
exp2_4 = exp2_2.query('시도명 != "세종특별자치시"')

In [295]:
exp2_4.at[600, '기초지자체'] = '경기도 용인시 기흥구'
exp2_4.at[1599, '기초지자체'] = '경상북도 경주시'
exp2_4.at[1600, '기초지자체'] = '경상북도 경주시'
exp2_4.at[2621, '기초지자체'] = '서울특별시 송파구'
exp2_4.at[2657, '기초지자체'] = '서울특별시 마포구'
exp2_4.at[3206, '기초지자체'] = '제주특별자치도 제주시'
exp2_4.at[3207, '기초지자체'] = '제주특별자치도 서귀포시'
exp2_4.at[3208, '기초지자체'] = '제주특별자치도 서귀포시'
exp2_4.at[3208, '기초지자체'] = '제주특별자치도 서귀포시'
exp2_4.at[3209, '기초지자체'] = '제주특별자치도 제주시'
exp2_4.at[3210, '기초지자체'] = '제주특별자치도 서귀포시'
exp2_4.at[3211, '기초지자체'] = '제주특별자치도 서귀포시'
exp2_4.at[3215, '기초지자체'] = '제주특별자치도 제주시'
exp2_4.at[3217, '기초지자체'] = '제주특별자치도 제주시'
exp2_4.at[3218, '기초지자체'] = '제주특별자치도 제주시'
exp2_4.at[3219, '기초지자체'] = '제주특별자치도 제주시'
exp2_4.at[3795, '기초지자체'] = '경기도 파주시'
exp2_4.at[3796, '기초지자체'] = '전라남도 해남군'

In [296]:
exp = pd.concat([exp1, exp2_1, exp2_3, exp2_4], ignore_index = True)

In [297]:
exp = exp.groupby('기초지자체').size().reset_index(name = '체험상품수')

In [298]:
exp.at[176, '체험상품수'] = 285
exp.at[177, '체험상품수'] = 163

In [299]:
del exp1, exp2, exp2_1, exp2_2, exp2_3, exp2_4

### 4-4. 키즈상품

In [300]:
kids = pd.read_csv('./야놀자/YN_REGNAL_KIDS_GOODS_202206.csv')

In [301]:
kids1 = kids[kids['KIDS_GOODS_ADDR'].isnull()]

In [302]:
kids1.at[490, '기초지자체'] = '경기도 용인시 기흥구'
kids1.at[2684, '기초지자체'] = '제주특별자치도 서귀포시'
kids1.at[2685, '기초지자체'] = '제주특별자치도 서귀포시'
kids1.at[2689, '기초지자체'] = '제주특별자치도 제주시'
kids1.at[2690, '기초지자체'] = '제주특별자치도 제주시'
kids1.at[3182, '기초지자체'] = '경기도 파주시'
kids1.at[3183, '기초지자체'] = '전라남도 해남군'

In [303]:
kids2 = kids[kids['KIDS_GOODS_ADDR'].isnull() == False]

In [304]:
kids2['KIDS_GOODS_ADDR'] = kids2['KIDS_GOODS_ADDR'].apply(convert_addr)

In [305]:
kids2['시도명'] = kids2['KIDS_GOODS_ADDR'].str.split(' ').str[0]
kids2['시군구명'] = kids2['KIDS_GOODS_ADDR'].str.split(' ').str[1]

In [306]:
kids2['기초지자체'] = kids2['시도명'] + ' ' + kids2['시군구명']

In [307]:
kids2_1 = kids2.query('기초지자체 in @standard')

In [308]:
kids2_2 = kids2.query('기초지자체 not in @standard')

In [309]:
kids2_3 = kids2_2.query('기초지자체 in @pp1')

In [310]:
kids2_3['기초지자체'] = kids2_3['시도명'] + ' ' + kids2_3['시군구명'] + ' ' + kids2_3['KIDS_GOODS_ADDR'].str.split(' ').str[2]

In [311]:
kids2_4 = kids2_2.query('기초지자체 not in @pp1')

In [312]:
for idx in kids2_4[kids2_4['시도명'] == '세종특별자치시'].index :
    kids2_4.at[idx, '기초지자체'] = '세종특별자치시'

In [313]:
kids2_4.at[1405, '기초지자체'] = '경상북도 경주시'
kids2_4.at[1406, '기초지자체'] = '경상북도 경주시'
kids2_4.at[2683, '기초지자체'] = '제주특별자치도 서귀포시'

In [314]:
kids = pd.concat([kids1, kids2_1, kids2_3, kids2_4], ignore_index = True)

In [315]:
kids = kids.groupby('기초지자체').size().reset_index(name = '키즈상품수')

In [316]:
del kids1, kids2, kids2_1, kids2_2, kids2_3, kids2_4

### 4-5. 사업장 일반

In [317]:
room = pd.read_csv('./야놀자/YN_LDGS_PROPERTY_INFO_202206.csv')

In [318]:
room['PROPERTY_LOAD_ADDR'] = room['PROPERTY_LOAD_ADDR'].apply(convert_addr)

In [319]:
room['PROPERTY_LOAD_ADDR'] = room['PROPERTY_LOAD_ADDR'].str.replace('인천광역시 남구 ', '인천광역시 미추홀구 ')

In [320]:
room['시도명'] = room['PROPERTY_LOAD_ADDR'].str.split(' ').str[0]

In [321]:
room['시군구명'] = room['PROPERTY_LOAD_ADDR'].str.split(' ').str[1]

In [322]:
room['기초지자체'] = room['시도명'] + ' ' + room['시군구명']

In [323]:
room1 = room.query('기초지자체 in @standard')

In [324]:
room2 = room.query('기초지자체 not in @standard')

In [325]:
room2_1 = room2.query('기초지자체 in @pp1')

In [326]:
room2_1['기초지자체'] = room2_1['시도명'] + ' ' + room2_1['시군구명'] + ' ' + room2_1['PROPERTY_LOAD_ADDR'].str.split(' ').str[2]

In [327]:
room2_1.at[19643, '기초지자체'] = '경기도 안산시 단원구'
room2_1.at[21236, '기초지자체'] = '경기도 용인시 처인구'

In [328]:
room2_2 = room2.query('기초지자체 not in @pp1')

In [329]:
for idx in room2_2[room2_2['시도명'].str.contains('세종')].index :
    room2_2.at[idx, '기초지자체'] = '세종특별자치시'

In [330]:
room2_3 = room2_2[room2_2['기초지자체'] == '세종특별자치시']

In [331]:
room2_4 = room2_2[room2_2['기초지자체'] != '세종특별자치시']

In [332]:
for idx in room2_4[room2_4['PROPERTY_LOAD_ADDR'].str.contains('제주시')].index :
    room2_4.at[idx, '기초지자체'] = '제주특별자치도 제주시'

In [333]:
for idx in room2_4[room2_4['PROPERTY_LOAD_ADDR'].str.contains('서귀포시')].index :
    room2_4.at[idx, '기초지자체'] = '제주특별자치도 서귀포시'

In [334]:
room2_4.at[4400, '기초지자체'] = '전라북도 전주시 완산구'
room2_4.at[16132, '기초지자체'] = '경상북도 경주시'
room2_4.at[16204, '기초지자체'] = '경상북도 경주시'
room2_4.at[16629, '기초지자체'] = '경기도 가평군'
room2_4.at[16694, '기초지자체'] = '강원도 홍천군'
room2_4.at[16838, '기초지자체'] = '강원도 양양군'
room2_4.at[17067, '기초지자체'] = '경기도 가평군'
room2_4.at[17157, '기초지자체'] = '경기도 가평군'
room2_4.at[17944, '기초지자체'] = '충청북도 단양군'
room2_4.at[18082, '기초지자체'] = '충청남도 보령시'
room2_4.at[18083, '기초지자체'] = '충청남도 보령시'
room2_4.at[18084, '기초지자체'] = '전라북도 전주시 완산구'
room2_4.at[18088, '기초지자체'] = '경상북도 안동시'
room2_4.at[18193, '기초지자체'] = '전라남도 담양군'
room2_4.at[18371, '기초지자체'] = '부산광역시 기장군'
room2_4.at[18929, '기초지자체'] = '서울특별시 중구'
room2_4.at[19366, '기초지자체'] = '인천광역시 강화군'
room2_4.at[19471, '기초지자체'] = '경기도 연천군'
room2_4.at[19630, '기초지자체'] = '인천광역시 강화군'
room2_4.at[19917, '기초지자체'] = '경상남도 밀양시'
room2_4.at[19957, '기초지자체'] = '충청남도 태안군'
room2_4.at[19972, '기초지자체'] = '강원도 강릉시'
room2_4.at[20450, '기초지자체'] = '경기도 안산시 단원구'
room2_4.at[21269, '기초지자체'] = '충청남도 태안군'
room2_4.at[21276, '기초지자체'] = '인천광역시 강화군'
room2_4.at[21294, '기초지자체'] = '전라북도 전주시 완산구'

In [335]:
room = pd.concat([room1, room2_1, room2_3, room2_4], ignore_index = True)

In [336]:
room = room.groupby('기초지자체').size().reset_index(name = '사업장수')

In [337]:
del room1, room2, room2_1, room2_2, room2_3, room2_4

### 4-6. 축제캠핑

In [338]:
fstvl = pd.read_csv('./야놀자/YN_REGNAL_FSTVL_ND_CAMP_GOODS_202206.csv')

In [339]:
fstvl = fstvl[fstvl['FSTVL_ND_CAMPING_GOODS_NM'].str.contains('2019|2020|2021') == False]

In [340]:
fstvl['FSTVL_ND_CAMPING_GOODS_ADDR'] = fstvl['FSTVL_ND_CAMPING_GOODS_ADDR'].apply(convert_addr)

In [341]:
fstvl['시도명'] = fstvl['FSTVL_ND_CAMPING_GOODS_ADDR'].str.split(' ').str[0]

In [342]:
fstvl['시군구명'] = fstvl['FSTVL_ND_CAMPING_GOODS_ADDR'].str.split(' ').str[1]

In [343]:
fstvl['기초지자체'] = fstvl['시도명'] + ' ' + fstvl['시군구명']

In [344]:
fstvl1 = fstvl.query('기초지자체 in @standard')

In [345]:
fstvl2 = fstvl.query('기초지자체 not in @standard')

In [346]:
fstvl2['기초지자체'] = fstvl2['시도명'] + ' ' + fstvl2['시군구명'] + ' ' + fstvl2['FSTVL_ND_CAMPING_GOODS_ADDR'].str.split(' ').str[2]

In [347]:
fstvl = pd.concat([fstvl1, fstvl2], ignore_index = True)

In [348]:
fstvl = fstvl.groupby('기초지자체').size().reset_index(name = '축제캠핑상품수')

In [349]:
del fstvl1, fstvl2

### 4-7. 테마파크

In [350]:
theme = pd.read_csv('./야놀자/YN_REGNAL_TMPARK_FCLTY_INFO_202206.csv')

In [351]:
theme.shape

(3693, 5)

In [352]:
theme.drop(3148, axis = 0, inplace = True)

In [353]:
theme1 = theme[theme['THEMEPARK_FCLTY_ADDR'].isnull()]

In [354]:
theme1.at[398, '기초지자체'] = '경기도 부천시'
theme1.at[470, '기초지자체'] = '경기도 안성시'
theme1.at[506, '기초지자체'] = '경기도 용인시 처인구'
theme1.at[507, '기초지자체'] = '경기도 용인시 처인구'
theme1.at[865, '기초지자체'] = '경기도 고양시 일산동구'
theme1.at[1501, '기초지자체'] = '서울특별시 강남구'
theme1.at[3150, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3151, '기초지자체'] = '제주특별자치도 제주시'
theme1.at[3152, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3153, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3154, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3155, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3156, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3157, '기초지자체'] = '제주특별자치도 제주시'
theme1.at[3158, '기초지자체'] = '제주특별자치도 서귀포시'
theme1.at[3163, '기초지자체'] = '제주특별자치도 제주시'
theme1.at[3164, '기초지자체'] = '제주특별자치도 제주시'
theme1.at[3165, '기초지자체'] = '제주특별자치도 제주시'
theme1.at[3692, '기초지자체'] = '전라남도 해남군'

In [355]:
theme2 = theme[theme['THEMEPARK_FCLTY_ADDR'].isnull() == False]

In [356]:
theme2['THEMEPARK_FCLTY_ADDR'] = theme2['THEMEPARK_FCLTY_ADDR'].apply(convert_addr)

In [357]:
theme2['시도명'] = theme2['THEMEPARK_FCLTY_ADDR'].str.split(' ').str[0]

In [358]:
theme2['시군구명'] = theme2['THEMEPARK_FCLTY_ADDR'].str.split(' ').str[1]

In [359]:
theme2['기초지자체'] = theme2['시도명'] + ' ' + theme2['시군구명']

In [360]:
theme2_1 = theme2.query('기초지자체 in @standard')

In [361]:
theme2_2 = theme2.query('기초지자체 not in @standard')

In [362]:
theme2_3 = theme2_2.query('기초지자체 in @pp1')

In [363]:
theme2_3['기초지자체'] = theme2_3['시도명'] + ' ' + theme2_3['시군구명'] + ' ' + theme2_3['THEMEPARK_FCLTY_ADDR'].str.split(' ').str[2]

In [364]:
theme2_4 = theme2_2.query('기초지자체 not in @pp1')

In [365]:
for idx in theme2_4[theme2_4['시도명'].str.contains('세종')].index :
    theme2_4.at[idx, '기초지자체'] = '세종특별자치시'

In [366]:
theme2_4.at[2944, '기초지자체'] = '인천광역시 미추홀구'
theme2_4.at[3149, '기초지자체'] = '제주특별자치도 서귀포시'
theme2_4.at[3161, '기초지자체'] = '제주특별자치도 서귀포시'
theme2_4.at[3162, '기초지자체'] = '제주특별자치도 서귀포시'
theme2_4.at[3168, '기초지자체'] = '제주특별자치도 제주시'

In [367]:
theme = pd.concat([theme1, theme2_1, theme2_3, theme2_4], ignore_index = True)

In [368]:
theme = theme.groupby('기초지자체').size().reset_index(name = '테마파크수')

In [369]:
theme.at[123, '테마파크수'] = 169
theme.at[122, '테마파크수'] = 312

In [370]:
del theme1, theme2, theme2_1, theme2_2, theme2_3, theme2_4

### 4-8. 워터파크

In [371]:
water = pd.read_csv('./야놀자/YN_REGNAL_WATERPARK_FCLTY_INFO_202206.csv')

In [372]:
water.at[113, 'WATERPARK_FCLTY_ADDR'] = '강원도 평창군'

In [373]:
water['WATERPARK_FCLTY_ADDR'] = water['WATERPARK_FCLTY_ADDR'].apply(convert_addr)

In [374]:
water['시도명'] = water['WATERPARK_FCLTY_ADDR'].str.split(' ').str[0]

In [375]:
water['시군구명'] = water['WATERPARK_FCLTY_ADDR'].str.split(' ').str[1]

In [376]:
water['기초지자체'] = water['시도명'] + ' ' + water['시군구명']

In [377]:
water1 = water.query('기초지자체 in @standard')

In [378]:
water2 = water.query('기초지자체 not in @standard')

In [379]:
water2['기초지자체'] = water2['시도명'] + ' ' + water2['시군구명'] + ' ' + water2['WATERPARK_FCLTY_ADDR'].str.split(' ').str[2]

In [380]:
water = pd.concat([water1, water2], ignore_index = True)

In [381]:
water = water.groupby('기초지자체').size().reset_index(name = '워터파크수')

In [382]:
del water1, water2

In [383]:
import gc

In [384]:
gc.collect()

103

### 4-9. 익스트림상품

In [385]:
extreme = pd.read_csv('./야놀자/YN_REGNAL_EXTR_GOODS_202206.csv')

In [386]:
extreme.at[25, 'EXTR_GOODS_ADDR'] = '강원도 양양군'
extreme.at[1533, 'EXTR_GOODS_ADDR'] = '전라남도 장성군'
extreme.at[1645, 'EXTR_GOODS_ADDR'] = '제주특별자치도 서귀포시'
extreme.at[1646, 'EXTR_GOODS_ADDR'] = '제주특별자치도 서귀포시'
extreme.at[1648, 'EXTR_GOODS_ADDR'] = '제주특별자치도 제주시'
extreme.at[1649, 'EXTR_GOODS_ADDR'] = '제주특별자치도 제주시'

In [387]:
extreme['EXTR_GOODS_ADDR'] = extreme['EXTR_GOODS_ADDR'].apply(convert_addr)

In [388]:
extreme['시도명'] = extreme['EXTR_GOODS_ADDR'].str.split(' ').str[0]

In [389]:
extreme['시군구명'] = extreme['EXTR_GOODS_ADDR'].str.split(' ').str[1]

In [390]:
extreme['기초지자체'] = extreme['시도명'] + ' ' + extreme['시군구명']

In [391]:
extreme1 = extreme.query('기초지자체 in @standard')

In [392]:
extreme2 = extreme.query('기초지자체 not in @standard')

In [393]:
extreme2_1 = extreme2.query('기초지자체 in @pp1')

In [394]:
extreme2_1['기초지자체'] = extreme2_1['시도명'] + ' ' + extreme2_1['시군구명'] + ' ' + extreme2_1['EXTR_GOODS_ADDR'].str.split(' ').str[2]

In [395]:
extreme2_2 = extreme2.query('기초지자체 not in @pp1')

In [396]:
extreme2_2.at[1405, '기초지자체'] = '경상북도 경주시'
extreme2_2.at[1406, '기초지자체'] = '경상북도 경주시'

In [397]:
extreme = pd.concat([extreme1, extreme2_1, extreme2_2], ignore_index = True)

In [398]:
extreme = extreme.groupby('기초지자체').size().reset_index(name = '익스트림상품수')

In [399]:
del extreme1, extreme2, extreme2_1, extreme2_2

### 4-10. 공연전시

In [400]:
exh = pd.read_csv('./야놀자/YN_REGNAL_PBLPRFR_ND_DSPY_GOODS_202206.csv')

In [401]:
exh.drop(570, axis = 0, inplace = True)

In [402]:
exh.at[255, 'PBLPRFR_ND_DSPY_GOODS_ADDR'] = '서울특별시 광진구'
exh.at[296, 'PBLPRFR_ND_DSPY_GOODS_ADDR'] = '서울특별시 서대문구'
exh.at[324, 'PBLPRFR_ND_DSPY_GOODS_ADDR'] = '서울특별시 종로구'
exh.at[325, 'PBLPRFR_ND_DSPY_GOODS_ADDR'] = '서울특별시 종로구'

In [403]:
exh['PBLPRFR_ND_DSPY_GOODS_ADDR'] = exh['PBLPRFR_ND_DSPY_GOODS_ADDR'].apply(convert_addr)

In [404]:
exh['시도명'] = exh['PBLPRFR_ND_DSPY_GOODS_ADDR'].str.split(' ').str[0]

In [405]:
exh['시군구명'] = exh['PBLPRFR_ND_DSPY_GOODS_ADDR'].str.split(' ').str[1]

In [406]:
exh['기초지자체'] = exh['시도명'] + ' ' + exh['시군구명']

In [407]:
exh1 = exh.query('기초지자체 in @standard')

In [408]:
exh2 = exh.query('기초지자체 not in @standard')

In [409]:
exh2_1 = exh2.query('기초지자체 in @pp1')

In [410]:
exh2_1['기초지자체'] = exh2_1['시도명'] + ' ' + exh2_1['시군구명'] + ' ' + exh2_1['PBLPRFR_ND_DSPY_GOODS_ADDR'].str.split(' ').str[2]

In [411]:
exh2_2 = exh2.query('기초지자체 not in @pp1')

In [412]:
exh2_2.at[489, '기초지자체'] = '세종특별자치시'
exh2_2.at[490, '기초지자체'] = '세종특별자치시'

In [413]:
exh = pd.concat([exh1, exh2_1, exh2_2], ignore_index = True)

In [414]:
exh = exh.groupby('기초지자체').size().reset_index(name = '공연전시상품수')

In [415]:
exh.at[76, '기초지자체'] = 122
exh.at[77, '기초지자체'] = 59

In [416]:
del exh1, exh2, exh2_1, exh2_2

In [417]:
gc.collect()

578

***
# 데이터 병합

In [418]:
data = pd.merge(population, visitors, how = 'left')
data = pd.merge(visitors, search, how = 'left')
data = pd.merge(data, consumption, how = 'left')
data = pd.merge(data, paid, how = 'left')
data = pd.merge(data, vg, how = 'left')
data = pd.merge(data, cg, how = 'left')
data = pd.merge(data, vs, how = 'left')

data = pd.merge(data, ml, how = 'left')
data = pd.merge(data, street, how = 'left')
data = pd.merge(data, parking, how = 'left')
data = pd.merge(data, food, how = 'left')
data = pd.merge(data, contest, how = 'left')
data = pd.merge(data, festival, how = 'left')
data = pd.merge(data, road, how = 'left')
data = pd.merge(data, forest, how = 'left')
data = pd.merge(data, park, how = 'left')
data = pd.merge(data, relics, how = 'left')
data = pd.merge(data, rural, how = 'left')

data = pd.merge(data, hobby, how = 'left')
data = pd.merge(data, theme, how = 'left')
data = pd.merge(data, room, how = 'left')
data = pd.merge(data, kids, how = 'left')
data = pd.merge(data, fstvl, how = 'left')
data = pd.merge(data, exh, how = 'left')
data = pd.merge(data, exp, how = 'left')
data = pd.merge(data, extreme, how = 'left')
data = pd.merge(data, water, how = 'left')
data = pd.merge(data, ynj_tour, how = 'left')

In [419]:
data.fillna(0, inplace = True)

In [420]:
data.to_csv('Final_Data.csv', index = False)